<a href="https://colab.research.google.com/github/vivianjjiang/ds2002/blob/main/FinanceAPIHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import pandas as pd
import json
import requests

Objective
In this assignment, you will:
1. Request stock data for two or more stock symbols.
2. Extract and display only the Stock Ticker, Company Name, and Current Market Price.
3. Allow the user to call another endpoint with a module and store the response in a
DataFrame.

Task 1: Fetch Basic Stock Data 10pts
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price.
Example Output:
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $175.23
Stock Ticker: MSFT, Company: Microsoft Corp., Current Market Price: $310.45

In [5]:
#get stock from the user
stock=input()

APPL,MSFT


In [6]:
apikey=""

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.get(url, headers=headers, params=querystring)

print(response.text)

{"quoteResponse":{"result":[{"language":"en-US","region":"US","quoteType":"MUTUALFUND","typeDisp":"Fund","quoteSourceName":"Delayed Quote","triggerable":false,"customPriceAlertConfidence":"NONE","marketState":"POST","corporateActions":[],"regularMarketTime":1561759658,"exchange":"YHD","exchangeTimezoneName":"America/New_York","exchangeTimezoneShortName":"EST","gmtOffSetMilliseconds":-18000000,"market":"us_market","esgPopulated":false,"hasPrePostMarketData":false,"priceHint":2,"fullExchangeName":"YHD","sourceInterval":15,"exchangeDataDelayedBy":0,"tradeable":false,"cryptoTradeable":false,"symbol":"APPL"},{"language":"en-US","region":"US","quoteType":"EQUITY","typeDisp":"Equity","quoteSourceName":"Nasdaq Real Time Price","triggerable":true,"customPriceAlertConfidence":"HIGH","currency":"USD","marketState":"POST","corporateActions":[],"postMarketTime":1740607811,"regularMarketTime":1740603600,"regularMarketChangePercent":0.45991886,"regularMarketPrice":399.73,"exchange":"NMS","messageBoar

In [ ]:
df = pd.DataFrame(response.json()['quoteResponse']['result'])
print(df[['symbol','shortName','regularMarketPrice']])


  symbol              shortName  regularMarketPrice
0   AAPL             Apple Inc.               247.1
1   MSFT  Microsoft Corporation               404.0


Task 2.1: Fetch Additional Data Using Modules 10pts
- Allow the user to choose a module from the Quote Summary Endpoint.
Find:
52 Week High
52 Week Low
Return on Assets (ROA)
- Make an API request using the chosen module.
- Convert the response into a Pandas DataFrame.
- Display the DataFrame.

In [19]:
module = input()

summaryDetail,financialData


In [20]:
url = "https://yfapi.net/v11/finance/quoteSummary/AAPL"

querystring = {
    "modules": module
}

headers = {
    'x-api-key': apikey
}

# headers = {'Authorization': f'Bearer {apikey}'}

response = requests.get(url, headers=headers, params=querystring)

# Pretty print the JSON response
print(json.dumps(response.json(), indent=4))

{
    "quoteSummary": {
        "result": [
            {
                "summaryDetail": {
                    "maxAge": 1,
                    "priceHint": {
                        "raw": 2,
                        "fmt": "2",
                        "longFmt": "2"
                    },
                    "previousClose": {
                        "raw": 247.04,
                        "fmt": "247.04"
                    },
                    "open": {
                        "raw": 244.33,
                        "fmt": "244.33"
                    },
                    "dayLow": {
                        "raw": 239.13,
                        "fmt": "239.13"
                    },
                    "dayHigh": {
                        "raw": 244.98,
                        "fmt": "244.98"
                    },
                    "regularMarketPreviousClose": {
                        "raw": 247.04,
                        "fmt": "247.04"
                    },
           

In [22]:
df = pd.DataFrame(response.json()['quoteSummary']['result'])
df_summary = pd.json_normalize(df['summaryDetail'])
df_financial = pd.json_normalize(df['financialData'])
df_merged = pd.concat([df_summary, df_financial], axis=1)
df_merged.rename(columns={"fiftyTwoWeekHigh.raw": "52 Week High", "fiftyTwoWeekLow.raw": "52 Week Low", "returnOnAssets.raw": "Return On Assets (ROA)"}, inplace=True)
print(df_merged[["52 Week High","52 Week Low", "Return On Assets (ROA)"]])

   52 Week High  52 Week Low  Return On Assets (ROA)
0         260.1       164.08                 0.22519


Task 2.2: Fetch Additional Data Using Modules 10pts
Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as
their 52 high and low.

In [23]:
trend_url = "https://yfapi.net/v1/finance/trending/US"
qs_url = "https://yfapi.net/v11/finance/quoteSummary"

In [24]:
def get_trending_stocks():
    headers = {'x-api-key': apikey}
    response = requests.get(trend_url, headers=headers)

    if response.status_code == 200:
        data = response.json().get("finance", {}).get("result", [])

        if not data:
            print("No trending stocks found.")
            return []

        # grab stock symbols from trending list
        return [item["symbol"] for item in data[0]["quotes"]]

    else:
        print("Failed to fetch trending stocks.")
        return []

def get_stock_details(symbols):
    stock_data = []

    for symbol in symbols:
        querystring = {"modules": "price,summaryDetail"}
        headers = {'x-api-key': apikey}
        response = requests.get(f"{qs_url}/{symbol}", headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json().get("quoteSummary", {}).get("result", [])

            if not data:
                print(f"No data found for {symbol}.")
                continue

            stock_info = {
                "Stock Ticker": symbol,
                "Company": data[0]["price"].get("longName", "N/A"),
                "Current Price": f"${data[0]['price'].get('regularMarketPrice', {}).get('raw', 'N/A')}",
                "52 Week High": data[0]["summaryDetail"].get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
                "52 Week Low": data[0]["summaryDetail"].get("fiftyTwoWeekLow", {}).get("raw", "N/A")
            }
            stock_data.append(stock_info)

        else:
            print(f"Failed to fetch details for {symbol}. Status Code: {response.status_code}")

    if stock_data:
        df = pd.DataFrame(stock_data)
        print(df)
    else:
        print("No valid data to display.")

In [25]:
trending_symbols = get_trending_stocks()

if trending_symbols:
    get_stock_details(trending_symbols)
else:
    print("No trending stocks to display.")

   Stock Ticker                               Company Current Price  \
0          NVDA                    NVIDIA Corporation       $131.28   
1           CRM                      Salesforce, Inc.       $307.33   
2          SNOW                        Snowflake Inc.       $166.19   
3          IONQ                            IonQ, Inc.        $29.93   
4          BYND                     Beyond Meat, Inc.         $3.56   
5           APP                  AppLovin Corporation        $331.0   
6          EBAY                             eBay Inc.        $69.14   
7            AI                           C3.ai, Inc.        $26.44   
8           NIO                              NIO Inc.         $4.72   
9          SNPS                        Synopsys, Inc.       $470.46   
10         MULN               Mullen Automotive, Inc.         $4.04   
11         NTNX                         Nutanix, Inc.        $69.35   
12         MARA                   MARA Holdings, Inc.        $12.45   
13    

Bonus: